In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# read in all the datasets
def read_clean(filename, date_vars = ["date"]):
    return pd.read_csv("../Data/" + filename, dtype={'fips': str}, parse_dates = date_vars)

cases = read_clean("NYT_clean.csv")
acs = read_clean("ACS Data.csv", None)
hospitals = read_clean("Hospitals_clean.csv").drop(columns=["collection_week"])
masks = read_clean("Masks_clean.csv")
mobility = read_clean("Mobility Data.csv")
IL_vaccine = read_clean("il_vaccine_clean.csv", ["Report_Date"]).drop(columns=["Unnamed: 0"]).drop_duplicates()
MO_vaccine = read_clean("MO_vaccine_clean.csv")
OH_vaccine = read_clean("oh_vaccine_clean.csv").drop(columns=["Unnamed: 0"])


In [3]:
# merge together all non-vaccine data
merged = cases.merge(acs.drop(columns=["county", "state"]), on=["fips"], how="left")\
              .merge(hospitals.drop(columns=["state"]), on=["fips", "date"], how="left")\
              .merge(masks.drop(columns=["county", "state"]), on=["fips", "date"], how="left")\
              .merge(mobility.drop(columns=["country", "state", "county"]), on=["fips", "date"], how="left")

In [4]:
# standardize vaccine data
IL_vaccine = IL_vaccine.rename(columns={"AdministeredCount": "cumulative doses",
                                        "Report_Date": "date"})
IL_vaccine['new doses'] = IL_vaccine.groupby(["state", "CountyName", "county", "fips"])['cumulative doses'].diff()


OH_vaccine['new doses'] = OH_vaccine["vaccines_started"] + OH_vaccine["vaccines_completed"]
OH_vaccine['new doses'] = OH_vaccine["vaccines_started"] + OH_vaccine["vaccines_completed"]
OH_vaccine["cumulative doses"] = OH_vaccine.sort_values(by=["state", "county", "fips", "date"])\
                                                       .groupby(['state', "county", "fips"])['new doses']\
                                                       .cumsum(axis=0)


# stack vaccine data
vaccines = IL_vaccine[["fips", "date", "new doses", "cumulative doses"]]\
            .append(OH_vaccine[["fips", "date", "new doses", "cumulative doses"]])\
            .append(MO_vaccine[["fips", "date", "new doses", "cumulative doses"]])

# merge onto rest of the data
merged = merged.merge(vaccines, on=["fips", "date"], how="left")

merged[["new doses", "cumulative doses"]] = merged[["new doses", "cumulative doses"]].fillna(0)

In [5]:
# look at a subset that should have all the data valued basically
merged[merged["date"] > '2020-12-01'].head()

,state,fips,county,date,cumulative_cases,cumulative_deaths,new_cases,new_deaths,new_cases_7avg,new_deaths_7avg,2weeksago_cases_7avg,2weeksago_deaths_7avg,total_pop,male,perc_male,female,perc_female,age_15_19,p_age_15_19,age_20_24,p_age_20_24,age_25_34,p_age_25_34,age_35_44,p_age_35_44,age_45_54,p_age_45_54,age_55_59,p_age_55_59,age_60_64,p_age_60_64,age_median,age_62over,p_age_62over,age_65over,p_age_65over,white,p_white,black,p_black,native,p_native,asian,p_asian,hawaiian,p_hawaiian,other_race,p_other_race,hispanic,p_hispanic,housing_units,below_50_pov,below_125_pov,below_150_pov,below_185_pov,below_200_pov,below_300_pov,below_400_pov,below_500_pov,below_pov,male_below_pov,female_below_pov,age_under14,p_under14,non_white,p_non_white,total_adult_hospitalizations,total_pediatric_hospitalizations,prev_day_adult_admit_7daysum,prev_day_adult_admit_18-19_7daysum,prev_day_adult_admit_20-29_7daysum,prev_day_adult_admit_30-39_7daysum,prev_day_adult_admit_40-49_7daysum,prev_day_adult_admit_50-59_7daysum,prev_day_adult_admit_60-69_7daysum,prev_day_adult_admit_70-79_7daysum,prev_day_adult_admit_80+_7daysum,prev_day_adult_admit_unknown_7daysum,mask_mandate,retail_rec,grocery_pharm,parks,transit,workplace,residential,new doses,cumulative doses
257,IL,17001,Adams,2020-12-02,4886,54.0,-15.0,4.0,33.0,1.0,114.0,1.0,66085,32276,48.8,33809,51.2,3679,5.6,3684,5.6,8060,12.2,7657,11.6,8105,12.3,4866,7.4,4251,6.4,41.5,15621,23.6,13186,20.0,61406,92.9,2731,4.1,297,0.4,527,0.8,14,0.0,113,0.2,1059,1.6,30192,3447,10417,13344,17583,19205,33041,43233,50182,8031,3605,4426,12597,19.1,4679,7.1,434.0,0.0,45.0,0.0,0.0,3.0,3.0,4.0,11.0,13.0,12.0,0.0,1.0,-14.0,-3.0,NaN,-32.0,-17.0,7.0,0.0,0.0
258,IL,17001,Adams,2020-12-03,4996,55.0,110.0,1.0,49.0,1.0,124.0,1.0,66085,32276,48.8,33809,51.2,3679,5.6,3684,5.6,8060,12.2,7657,11.6,8105,12.3,4866,7.4,4251,6.4,41.5,15621,23.6,13186,20.0,61406,92.9,2731,4.1,297,0.4,527,0.8,14,0.0,113,0.2,1059,1.6,30192,3447,10417,13344,17583,19205,33041,43233,50182,8031,3605,4426,12597,19.1,4679,7.1,434.0,0.0,45.0,0.0,0.0,3.0,3.0,4.0,11.0,13.0,12.0,0.0,1.0,-18.0,5.0,NaN,-43.0,-20.0,7.0,0.0,0.0
259,IL,17001,Adams,2020-12-04,5049,56.0,53.0,1.0,57.0,1.0,110.0,1.0,66085,32276,48.8,33809,51.2,3679,5.6,3684,5.6,8060,12.2,7657,11.6,8105,12.3,4866,7.4,4251,6.4,41.5,15621,23.6,13186,20.0,61406,92.9,2731,4.1,297,0.4,527,0.8,14,0.0,113,0.2,1059,1.6,30192,3447,10417,13344,17583,19205,33041,43233,50182,8031,3605,4426,12597,19.1,4679,7.1,340.0,0.0,45.0,0.0,3.0,3.0,0.0,3.0,12.0,20.0,7.0,0.0,1.0,-18.0,-6.0,NaN,-37.0,-14.0,7.0,0.0,0.0
260,IL,17001,Adams,2020-12-05,5038,57.0,-11.0,1.0,44.0,1.0,124.0,1.0,66085,32276,48.8,33809,51.2,3679,5.6,3684,5.6,8060,12.2,7657,11.6,8105,12.3,4866,7.4,4251,6.4,41.5,15621,23.6,13186,20.0,61406,92.9,2731,4.1,297,0.4,527,0.8,14,0.0,113,0.2,1059,1.6,30192,3447,10417,13344,17583,19205,33041,43233,50182,8031,3605,4426,12597,19.1,4679,7.1,340.0,0.0,45.0,0.0,3.0,3.0,0.0,3.0,12.0,20.0,7.0,0.0,1.0,-26.0,0.0,NaN,-40.0,0.0,3.0,0.0,0.0
261,IL,17001,Adams,2020-12-06,5074,57.0,36.0,0.0,44.0,1.0,110.0,1.0,66085,32276,48.8,33809,51.2,3679,5.6,3684,5.6,8060,12.2,7657,11.6,8105,12.3,4866,7.4,4251,6.4,41.5,15621,23.6,13186,20.0,61406,92.9,2731,4.1,297,0.4,527,0.8,14,0.0,113,0.2,1059,1.6,30192,3447,10417,13344,17583,19205,33041,43233,50182,8031,3605,4426,12597,19.1,4679,7.1,340.0,0.0,45.0,0.0,3.0,3.0,0.0,3.0,12.0,20.0,7.0,0.0,1.0,-18.0,0.0,NaN,NaN,-5.0,3.0,0.0,0.0


In [6]:
###############################
# Treat NAs for hospital data #
###############################

# filling na's with 0 for data points after 7-31-2020 because that's when the data should be complete
# most counties that had missing values just didn't have any hospitals in the dataset associated with it, and
# I spot-checked a few and it seemed to be counties without large hospitals.  Therefore, let's fill those in with 0's
# Also manually checked the two counties that had sporadic missings, but the surrounding dates (around the missing 
# dates) were all basically 0 so I feel okay filling in those with 0s
date_mask = (merged["date"]>='7-31-2020')
numerical_hosp_variables = hospitals.columns.tolist()[3:]
merged.loc[date_mask, numerical_hosp_variables] = merged.loc[date_mask, numerical_hosp_variables].fillna(0)

merged[date_mask].describe()

,cumulative_cases,cumulative_deaths,new_cases,new_deaths,new_cases_7avg,new_deaths_7avg,2weeksago_cases_7avg,2weeksago_deaths_7avg,total_pop,male,perc_male,female,perc_female,age_15_19,p_age_15_19,age_20_24,p_age_20_24,age_25_34,p_age_25_34,age_35_44,p_age_35_44,age_45_54,p_age_45_54,age_55_59,p_age_55_59,age_60_64,p_age_60_64,age_median,age_62over,p_age_62over,age_65over,p_age_65over,white,p_white,black,p_black,native,p_native,asian,p_asian,hawaiian,p_hawaiian,other_race,p_other_race,hispanic,p_hispanic,housing_units,below_50_pov,below_125_pov,below_150_pov,below_185_pov,below_200_pov,below_300_pov,below_400_pov,below_500_pov,below_pov,male_below_pov,female_below_pov,age_under14,p_under14,non_white,p_non_white,total_adult_hospitalizations,total_pediatric_hospitalizations,prev_day_adult_admit_7daysum,prev_day_adult_admit_18-19_7daysum,prev_day_adult_admit_20-29_7daysum,prev_day_adult_admit_30-39_7daysum,prev_day_adult_admit_40-49_7daysum,prev_day_adult_admit_50-59_7daysum,prev_day_adult_admit_60-69_7daysum,prev_day_adult_admit_70-79_7daysum,prev_day_adult_admit_80+_7daysum,prev_day_adult_admit_unknown_7daysum,mask_mandate,retail_rec,grocery_pharm,parks,transit,workplace,residential,new doses,cumulative doses
count,99168.000000,99168.000000,99168.000000,99168.000000,99168.000000,99168.000000,99168.000000,99168.000000,9.916800e+04,9.916800e+04,99168.000000,9.916800e+04,99168.000000,99168.000000,99168.000000,99168.000000,99168.000000,99168.000000,99168.000000,99168.000000,99168.000000,99168.000000,99168.000000,99168.000000,99168.00000,99168.000000,99168.000000,99168.000000,99168.000000,99168.000000,99168.000000,99168.000000,9.916800e+04,99168.000000,9.916800e+04,99168.000000,99168.000000,99168.000000,99168.000000,99168.000000,99168.000000,99168.000000,99168.000000,99168.000000,9.916800e+04,99168.000000,9.916800e+04,99168.000000,99168.000000,9.916800e+04,9.916800e+04,9.916800e+04,9.916800e+04,9.916800e+04,9.916800e+04,99168.000000,99168.000000,99168.000000,99168.000000,99168.000000,9.916800e+04,99168.000000,99168.000000,99168.000000,99168.000000,99168.000000,99168.000000,99168.000000,99168.000000,99168.000000,99168.000000,99168.000000,99168.000000,99168.000000,91368.000000,53632.000000,47455.000000,15347.000000,27859.000000,82335.000000,55364.000000,99168.000000,9.916800e+04
mean,5161.793825,104.394694,33.953554,0.512000,33.757997,0.459110,32.776642,0.457083,1.050990e+05,5.160025e+04,49.993418,5.349870e+04,50.007096,6909.538813,6.417592,7114.839626,6.134859,13964.800319,11.455606,12823.376573,11.418067,13680.147951,12.859363,7350.418593,7.45083,6869.509600,7.171098,42.188211,20984.880818,23.486382,17002.676206,19.274425,8.152466e+04,91.127910,1.391686e+04,4.266934,356.339595,0.581464,3588.802487,1.003384,53.487869,0.050869,2758.465483,0.800889,8.807646e+03,3.603859,4.659869e+04,6246.981698,18218.865904,2.244630e+04,2.872043e+04,3.138698e+04,4.833279e+04,6.300262e+04,7.436655e+04,13911.779626,6176.062389,7735.717237,19383.643080,17.828151,2.357429e+04,8.872090,127.662976,1.504195,17.498780,0.181823,1.048574,1.364674,1.820880,2.981254,4.034971,4.402216,4.084221,0.586782,0.699435,-6.652577,0.065999,26.913077,-8.161958,-19.373061,6.509266,142.627420,5.642187e+03
std,20168.203390,459.246271,143.899082,3.197604,125.964387,2.105418,123.692629,2.104367,3.216132e+05,1.560347e+05,1.863287,1.655913e+05,1.863478,19973.800395,1.200686,21437.383488,2.370029,50292.543435,1.700332,42172.381126,1.233613,41405.299756,1.010909,21172.486409,0.87404,19524.374256,1.156920,4.772353,58259.916932,4.609331,47013.143048,3.969242,1.961639e+05,8.069725,7.794258e+04,6.301037,970.268476,1.346136,21655.225523,1.453973,192.360471,0.161156,25964.455046,1.293008,6.862619e+04,3.775163,1.377733e+05,21346.518588,60804.100464,7.380772e+04,9.261169e+04,1.004425e+05,1.481469e+05,1.874362e+05,2.191494e+05,46757.102437,20581.282852,26187.276364,59630.191257,2.143485,1.305233e+05,8.069725,542.473721,8.901669,63.109257,0.995358,4.450211,5.632822,6.990652,10.866424,13.675650,

In [7]:
###############################
# Treat NAs for mask data #
###############################

# since mask mandate data ends 3-22-2021 and our covid data goes on until 4-11-2021 (as of right now), let's forward
# fill NA's after 3-22-2021 (I've hand checked and the mask mandates have not changed for our three states)
mask_date_mask = (merged["date"] >= '3-22-2021')
merged.loc[mask_date_mask, 'mask_mandate'] = merged.loc[mask_date_mask, 'mask_mandate'].fillna(method='ffill', 
                                                                                               limit = 20)
merged.loc[mask_date_mask & merged["mask_mandate"].isnull()].head()

,state,fips,county,date,cumulative_cases,cumulative_deaths,new_cases,new_deaths,new_cases_7avg,new_deaths_7avg,2weeksago_cases_7avg,2weeksago_deaths_7avg,total_pop,male,perc_male,female,perc_female,age_15_19,p_age_15_19,age_20_24,p_age_20_24,age_25_34,p_age_25_34,age_35_44,p_age_35_44,age_45_54,p_age_45_54,age_55_59,p_age_55_59,age_60_64,p_age_60_64,age_median,age_62over,p_age_62over,age_65over,p_age_65over,white,p_white,black,p_black,native,p_native,asian,p_asian,hawaiian,p_hawaiian,other_race,p_other_race,hispanic,p_hispanic,housing_units,below_50_pov,below_125_pov,below_150_pov,below_185_pov,below_200_pov,below_300_pov,below_400_pov,below_500_pov,below_pov,male_below_pov,female_below_pov,age_under14,p_under14,non_white,p_non_white,total_adult_hospitalizations,total_pediatric_hospitalizations,prev_day_adult_admit_7daysum,prev_day_adult_admit_18-19_7daysum,prev_day_adult_admit_20-29_7daysum,prev_day_adult_admit_30-39_7daysum,prev_day_adult_admit_40-49_7daysum,prev_day_adult_admit_50-59_7daysum,prev_day_adult_admit_60-69_7daysum,prev_day_adult_admit_70-79_7daysum,prev_day_adult_admit_80+_7daysum,prev_day_adult_admit_unknown_7daysum,mask_mandate,retail_rec,grocery_pharm,parks,transit,workplace,residential,new doses,cumulative doses
87700,MO,29095,Jackson,2021-04-02,79956,964.0,97.0,1.0,70.0,1.0,91.0,1.0,696216,336640,48.4,359576,51.6,41914,6.0,42651,6.1,107785,15.5,87114,12.5,86454,12.4,47743,6.9,42296,6.1,36.8,126374,18.2,101918,14.6,466132,67.0,162382,23.3,2458,0.4,12821,1.8,2250,0.3,27412,3.9,62514,9.0,323157,45773,132399,160102,205135,224782,344401,446927,522386,100359,43357,57002,138341,19.9,230084,33.0,454.0,12.0,65.0,0.0,6.0,13.0,15.0,20.0,12.0,16.0,9.0,0.0,NaN,5.0,7.0,57.0,-3.0,-27.0,5.0,4083.0,157910.0
87701,MO,29095,Jackson,2021-04-02,79956,964.0,97.0,1.0,70.0,1.0,91.0,1.0,696216,336640,48.4,359576,51.6,41914,6.0,42651,6.1,107785,15.5,87114,12.5,86454,12.4,47743,6.9,42296,6.1,36.8,126374,18.2,101918,14.6,466132,67.0,162382,23.3,2458,0.4,12821,1.8,2250,0.3,27412,3.9,62514,9.0,323157,45773,132399,160102,205135,224782,344401,446927,522386,100359,43357,57002,138341,19.9,230084,33.0,454.0,12.0,65.0,0.0,6.0,13.0,15.0,20.0,12.0,16.0,9.0,0.0,NaN,5.0,7.0,57.0,-3.0,-27.0,5.0,5849.0,210773.0
87702,MO,29095,Jackson,2021-04-03,80046,966.0,90.0,2.0,73.0,2.0,86.0,1.0,696216,336640,48.4,359576,51.6,41914,6.0,42651,6.1,107785,15.5,87114,12.5,86454,12.4,47743,6.9,42296,6.1,36.8,126374,18.2,101918,14.6,466132,67.0,162382,23.3,2458,0.4,12821,1.8,2250,0.3,27412,3.9,62514,9.0,323157,45773,132399,160102,205135,224782,344401,446927,522386,100359,43357,57002,138341,19.9,230084,33.0,454.0,12.0,65.0,0.0,6.0,13.0,15.0,20.0,12.0,16.0,9.0,0.0,NaN,-10.0,16.0,33.0,-11.0,-12.0,-1.0,1178.0,159088.0
87703,MO,29095,Jackson,2021-04-03,80046,966.0,90.0,2.0,73.0,2.0,86.0,1.0,696216,336640,48.4,359576,51.6,41914,6.0,42651,6.1,107785,15.5,87114,12.5,86454,12.4,47743,6.9,42296,6.1,36.8,126374,18.2,101918,14.6,466132,67.0,162382,23.3,2458,0.4,12821,1.8,2250,0.3,27412,3.9,62514,9.0,323157,45773,132399,160102,205135,224782,344401,446927,522386,100359,43357,57002,138341,19.9,230084,33.0,454.0,12.0,65.0,0.0,6.0,13.0,15.0,20.0,12.0,16.0,9.0,0.0,NaN,-10.0,16.0,33.0,-11.0,-12.0,-1.0,1878.0,212651.0
87704,MO,29095,Jackson,2021-04-04,80099,966.0,53.0,0.0,72.0,2.0,83.0,1.0,696216,336640,48.4,359576,51.6,41914,6.0,42651,6.1,107785,15.5,87114,12.5,86454,12.4,47743,6.9,42296,6.1,36.8,126374,18.2,101918,14.6,466132,67.0,162382,23.3,2458,0.4,12821,1.8,2250,0.3,27412,3.9,62514,9.0,323157,45773,132399,160102,205135,224782,344401,446927,522386,100359,43357,57002,138341,19.9,230084,33.0,454.0,12.0,65.0,0.0,6.0,13.0,15.0,20.0,12.0,16.0,9.0,0.0,NaN,-26.0,3.0,36.0,-1.0,-22.0,-1.0,399.0,159487.0


In [8]:
# drop MI data for now since we don't have vaccination data
merged = merged[merged["state"]!="MI"]

In [9]:
# export dataset
merged.to_csv("../Data/merged.csv", index=False)